Load crypot config

In [ ]:
import json

config_file = "../crypot.json"
config = json.load(open(config_file))
host = config['database']['host']
port = config['database']['port']
dbname = config['database']['name']
username = config['database']['username']
password = config['database']['password']
print(f"host:{host}")
print(f"port:{port}")
print(f"dbname:{dbname}")
print(f"username:{username}")
print(f"password:{password}")

Connect to db

In [5]:
import psycopg2
conn_string = f"host={host} port={port} user={username} dbname={dbname} password={password}"
conn = psycopg2.connect(conn_string)
print("Connection established")

Connection established


Get klines for Binance:BTCUSDT

In [6]:
cursor = conn.cursor()

_2018 = 1514764800000
_2021 = 1609459200000
_2022 = 1640995200000
_now = 1643743029000
epoch = 1645903224000
symbol = "BTCUSDT"
interval = 1 #mins

fromval = _2018
toval = _now

table_name = 'exchanges.binance_klines'
query_cmd = f'''
        WITH series AS (
            SELECT generate_series({fromval} - MOD({fromval}, {interval * 60000}), {toval} - MOD({toval}, {interval * 60000}), {interval * 60000}) AS r_from
        )
        SELECT 
            r_from as open_time, \n
            (SELECT open_price FROM {table_name} WHERE symbol = '{symbol}' AND open_time BETWEEN r_from AND r_from + {interval * 60000} ORDER BY open_time ASC LIMIT 1) as open_price,
            (SELECT close_price FROM {table_name} WHERE symbol = '{symbol}' AND open_time BETWEEN r_from AND r_from + {interval * 60000} ORDER BY open_time DESC LIMIT 1) as close_price,
            (SELECT MIN(low_price) FROM {table_name} WHERE symbol = '{symbol}' AND open_time BETWEEN r_from AND r_from + {interval * 60000}) as low_price,
            (SELECT MAX(high_price) FROM {table_name} WHERE symbol = '{symbol}' AND open_time BETWEEN r_from AND r_from + {interval * 60000}) as high_price,
            (SELECT SUM(base_volume) FROM {table_name} WHERE symbol = '{symbol}' AND open_time BETWEEN r_from AND r_from + {interval * 60000}) as base_volume,
            (SELECT MAX(quote_volume) FROM {table_name} WHERE symbol = '{symbol}' AND open_time BETWEEN r_from AND r_from + {interval * 60000}) as quote_volume
        FROM series;'''
        
cursor.execute(query_cmd)
rows = cursor.fetchall()

conn.commit()
cursor.close()

print(f"returned {len(rows)} rows")

returned 2149638 rows


In [52]:
import matplotlib.pyplot as plt
import pandas as pd
import plotly.graph_objects as go

ohlc = pd.DataFrame(rows, columns=[
                    'Date', 'Open', 'Close', 'Low', 'High', 'BaseVolume', 'QuoteVolume'])
ohlc.to_csv(f"{symbol}.csv.gz", index=False, compression='gzip')
ohlc['DateTime'] = pd.to_datetime(ohlc['Date'], unit='ms')

#trace1 = {
#    'x': ohlc['DateTime'],
#    'open': ohlc['Open'],
#    'close': ohlc['Close'],
#    'high':  ohlc['High'],
#    'low':  ohlc['Low'],
#    'type': 'candlestick',
#    'name': symbol,
#    'showlegend': True
#}
#
#fig = go.Figure(data=[trace1])
#
#fig.update_layout(
#    height=600
#)
#fig.show()
#